In [7]:
import pandas as pd
from nesta_ds_utils.loading_saving import S3

from discovery_child_development import PROJECT_DIR, S3_BUCKET
from discovery_child_development.utils.utils import get_yaml_config

In [12]:
CONFIG = get_yaml_config(PROJECT_DIR / "discovery_child_development/pipeline/labelling/taxonomy_v2/config.yaml")

In [19]:
df = S3.download_obj(
    bucket=S3_BUCKET,
    path_from=CONFIG["s3_directory"] + "testing_new" + ".parquet",
    download_as="dataframe"
)

In [24]:
i = i+1
print(df.iloc[i]["prediction"])
print(df.iloc[i]["text"])

['Technology (general)' 'Cognitive development' 'Data' 'Neuroscience']
A systematic survey of face stimuli used in psychological research 2000–2020. For decades, psychology has relied on highly standardized images to understand how people respond to faces. Many of these stimuli are rigorously generated and supported by excellent normative data; as such, they have played an important role in the development of face science. However, there is now clear evidence that testing with ambient images (i.e., naturalistic images “in the wild”) and including expressions that are spontaneous can lead to new and important insights. To precisely quantify the extent to which our current knowledge base has relied on standardized and posed stimuli, we systematically surveyed the face stimuli used in 12 key journals in this field across 2000–2020 (N = 3374 articles). Although a small number of posed expression databases continue to dominate the literature, the use of spontaneous expressions seems to be i